— tức là sử dụng mạng neural (Neural Networks) để dự đoán giá trị liên tục, thay vì phân loại.

Đây là bước mở đường vào Deep Learning cho hồi quy, và cực kỳ quan trọng trong các bài toán như: dự báo thời tiết, dự đoán giá cổ phiếu, ước lượng năng lượng tiêu thụ, v.v.

Mình sẽ trình bày theo đúng phong cách bạn yêu thích:

🔹 1. Nguyên lý hoạt động (không công thức nặng!)
🎯 Mục tiêu:
Dự đoán một số thực (giá nhà, nhiệt độ, doanh thu...) bằng cách học biểu diễn phi tuyến phức tạp từ dữ liệu, thông qua nhiều lớp nơ-ron nhân tạo.

🧠 Ý tưởng chính — “Học từ đơn giản đến phức tạp, rồi đưa ra con số”
Đầu vào: các đặc trưng (diện tích, số phòng, vị trí…)
Các lớp ẩn: học đặc trưng ngày càng trừu tượng
Lớp 1: kết hợp tuyến tính → áp hàm kích hoạt (ReLU)
Lớp 2: kết hợp kết quả lớp 1 → lại áp ReLU
...
Đầu ra: một nơ-ron duy nhất, không có hàm kích hoạt phi tuyến (hoặc dùng hàm identity) → ra giá trị thực
💡 Khác với phân loại (dùng softmax/sigmoid để ra xác suất),
hồi quy bằng mạng neural → lớp cuối cùng là tuyến tính. 

🔁 Quá trình học:
Forward pass: đưa dữ liệu qua mạng → ra dự đoán số thực.
Tính lỗi: thường dùng MSE (Mean Squared Error).
Backpropagation: lan truyền lỗi ngược → cập nhật trọng số.
Lặp lại → mạng ngày càng dự đoán chính xác hơn.
✅ Ưu điểm:
Mạnh với quan hệ phi tuyến phức tạp.
Tự động học đặc trưng — không cần thiết kế tay.
Mở rộng tốt: có thể thêm lớp, nơ-ron, hoặc chuyển sang CNN/RNN nếu cần.
❌ Hạn chế:
Cần nhiều dữ liệu để học tốt.
Tốn tài nguyên (GPU, thời gian huấn luyện).
Dễ overfit nếu không kiểm soát (dùng dropout, early stopping…).
“Hộp đen”: khó giải thích tại sao ra con số đó.


🔹 3. Khi nào dùng Neural Network Regressor?
Dữ liệu
rất lớn
(hàng chục nghìn mẫu trở lên)
Dữ liệu
nhỏ
(< 1000 mẫu) → dùng
Random Forest
hoặc
SVR
Mối quan hệ
phi tuyến cực kỳ phức tạp
Mối quan hệ
gần tuyến tính
→ Linear/Ridge/Lasso đủ
Bạn có
GPU
và
thời gian huấn luyện
Cần
kết quả nhanh, đơn giản
Bài toán
ảnh, chuỗi, văn bản
(khi mở rộng thành CNN/RNN)
Dữ liệu
bảng đơn giản
(Excel-style) → RF thường đủ và giải thích được

🎯 So sánh nhanh với các mô hình hồi quy khác:
Linear/Ridge/Lasso
✅ Tốt
✅ Tốt
❌ Kém
✅ Rất tốt
⚡ Rất nhanh
Decision Tree
✅ Tốt
✅ Tốt
✅ Tốt
✅ Tốt
⚡ Nhanh
Random Forest
✅✅ Rất tốt
✅ Tốt
✅✅ Rất tốt
⚠️ Trung bình
⚡ Nhanh
SVR
✅ Tốt
❌ Kém
✅ Tốt
❌ Kém
🐢 Chậm
Neural Network
❌ Kém
✅✅ Rất tốt
✅✅✅ Xuất sắc
❌ Hộp đen
🐢 Chậm (nếu không có GPU)

In [2]:
import numpy as np

class SimpleNeuralNetworkRegressor:
    def __init__(self, input_size, hidden_size, learning_rate=0.01):
        self.lr = learning_rate
        # Khởi tạo trọng số ngẫu nhiên
        self.W1 = np.random.randn(input_size, hidden_size) * 0.01
        self.b1 = np.zeros((1, hidden_size))
        self.W2 = np.random.randn(hidden_size, 1) * 0.01  # đầu ra: 1 giá trị
        self.b2 = np.zeros((1, 1))

    def _relu(self, x):
        return np.maximum(0, x)

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self._relu(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        return self.z2  # KHÔNG dùng softmax/sigmoid → giữ nguyên giá trị thực

    def backward(self, X, y_true, y_pred):
        m = X.shape[0]
        # Gradient tại đầu ra (với MSE loss)
        dz2 = (2 / m) * (y_pred - y_true)  # đạo hàm của MSE
        dW2 = np.dot(self.a1.T, dz2)
        db2 = np.sum(dz2, axis=0, keepdims=True)

        # Gradient tại lớp ẩn
        da1 = np.dot(dz2, self.W2.T)
        dz1 = da1 * (self.z1 > 0)  # đạo hàm ReLU
        dW1 = np.dot(X.T, dz1)
        db1 = np.sum(dz1, axis=0, keepdims=True)

        # Cập nhật
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1

    def fit(self, X, y, epochs=1000):
        y = y.reshape(-1, 1)  # đảm bảo shape (n, 1)
        for epoch in range(epochs):
            y_pred = self.forward(X)
            loss = np.mean((y_pred - y) ** 2)  # MSE
            self.backward(X, y, y_pred)
            if epoch % 200 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict(self, X):
        return self.forward(X).flatten()

# --- Thử nghiệm ---
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Tạo dữ liệu
X, y = make_regression(n_samples=500, n_features=10, noise=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Chuẩn hóa (giúp mạng học ổn định)
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()

# Huấn luyện mạng neural hồi quy tự code
nn_reg = SimpleNeuralNetworkRegressor(
    input_size=10, 
    hidden_size=16, 
    learning_rate=0.01
)
nn_reg.fit(X_train, y_train, epochs=1000)

# Dự đoán và hoàn ngược chuẩn hóa
y_pred_scaled = nn_reg.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

# Đánh giá
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"\n✅ MSE (Neural Network Regressor tự code): {mse:.2f}")
print(f"✅ R²: {r2:.4f}")

# So sánh với sklearn MLPRegressor
from sklearn.neural_network import MLPRegressor
sk_nn = MLPRegressor(
    hidden_layer_sizes=(16,), 
    activation='relu', 
    solver='adam', 
    max_iter=1000, 
    random_state=42
)
sk_nn.fit(X_train, y_train)
sk_pred_scaled = sk_nn.predict(X_test)
sk_pred = scaler_y.inverse_transform(sk_pred_scaled.reshape(-1, 1)).flatten()
sk_r2 = r2_score(y_test, sk_pred)
print(f"🔍 Sklearn MLPRegressor R²: {sk_r2:.4f}")

Epoch 0, Loss: 0.9999
Epoch 200, Loss: 0.9384
Epoch 400, Loss: 0.0653
Epoch 600, Loss: 0.0328
Epoch 800, Loss: 0.0318

✅ MSE (Neural Network Regressor tự code): 766.04
✅ R²: 0.9628
🔍 Sklearn MLPRegressor R²: 0.9312
